In [0]:
CSV_PATH = './data/'

In [3]:
import tensorflow as tf

TensorFlow 2.x selected.


In [0]:
from tqdm import tqdm
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, StratifiedKFold

In [5]:
train_df = pd.read_csv(os.path.join(CSV_PATH, 'train.csv'))
test_df = pd.read_csv(os.path.join(CSV_PATH, 'test.csv'))
print("Train shape : ",train_df.shape)
print("Test shape : ",test_df.shape)

Train shape :  (1306122, 3)
Test shape :  (375806, 2)


In [0]:
train_X = train_df["question_text"].fillna(" ")
test_X = test_df["question_text"].fillna(" ")
train_y = train_df['target'].values
bool_train_labels = train_y != 0
text_list = pd.concat([train_X,test_X])

In [0]:
def clean_text(x):

    x = str(x)
    for punct in "/-":
        x = x.replace(punct, ' ')
    for punct in '&':
        x = x.replace(punct, f' {punct} ')
    for punct in '?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~' + '“”’':
        x = x.replace(punct, '')
    return x

In [0]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()
from nltk.stem.lancaster import LancasterStemmer
lc = LancasterStemmer()
from nltk.stem import SnowballStemmer
sb = SnowballStemmer("english")

text_list = text_list.apply(lambda x: clean_text(x))
text_list = text_list.apply(lambda x: ps.stem(x))
text_list = text_list.apply(lambda x: lc.stem(x))
text_list = text_list.apply(lambda x: sb.stem(x))
text_list = list(text_list.apply(lambda x: x.split()).values)

In [0]:
def build_vocab(sentences, verbose =  True):
    """
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
    vocab = {}
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

In [0]:
import operator

def check_converge(vocab,embedding_index):
    iv={}
    oov={}
    iv_acc=0
    oov_acc=0
    for word in vocab:
        try:
            iv[word] = embedding_index[word]
            iv_acc += vocab[word]
            
        except:
            oov[word] = vocab[word]
            oov_acc += vocab[word]
            
    print("{:.2%}".format((len(iv)/len(vocab))))
    print("{:.2%}".format(iv_acc/(iv_acc+oov_acc)))

    
    sorted_x = sorted(oov.items(), key=operator.itemgetter(1))[::-1]
    return sorted_x

In [11]:
vocab = build_vocab(text_list)

100%|██████████| 1681928/1681928 [00:07<00:00, 230213.13it/s]


In [0]:
from six.moves.urllib.request import urlretrieve
from progressbar import Bar, ETA, FileTransferSpeed, ProgressBar, Percentage, RotatingMarker

In [13]:
#This code snippet is inspired by https://github.com/chakki-works/chakin


ROOT_DIR = './'
url = 'http://nlp.stanford.edu/data/glove.840B.300d.zip'
file_name = url.split('/')[-1]
save_path = os.path.join(ROOT_DIR, file_name)

widgets = ['Test: ', Percentage(), ' ', Bar(marker=RotatingMarker()), ' ', ETA(), ' ', FileTransferSpeed()]
pbar = ProgressBar(widgets=widgets)

def dlProgress(count, blockSize, totalSize):
    if pbar.max_value is None:
        pbar.max_value = totalSize
        pbar.start()

    pbar.update(min(count * blockSize, totalSize))

path, _ = urlretrieve(url, save_path, reporthook=dlProgress)
pbar.finish()

Test: 100% ||                                      | Time:  0:16:56   2.0 MiB/s


In [14]:
!unzip ./glove.840B.300d.zip

Archive:  ./glove.840B.300d.zip
  inflating: glove.840B.300d.txt     


In [0]:
EMBEDDING_FILE = './glove.840B.300d.txt' #fill in the file path of pre-trained embedding file you want to check oov test of
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

In [19]:
oov = check_converge(vocab, embeddings_index)

53.50%
98.78%
